## Creating a Language Model from scratch completely Locally

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu' #using cpu due to apple silicon restrictions
print(device)
block_size = 8
batch_size = 4

cpu


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '#', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


#### Setting encoders and decoders and using tensors to store data

In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [6]:
# example
print(encode('hello'))
print(decode(encode('hello')))

[62, 59, 66, 66, 69]
hello


#### Splitting data into training and validation sets

In [10]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# function to get inputs and targets


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)  # for cuda(not available on mac)
    return x, y


x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([158779,  72367,  58131, 173629])
inputs:
tensor([[85,  1, 73, 55, 63, 58,  1, 74],
        [72, 66, 59, 74,  1, 70, 69, 70],
        [68, 61, 66, 79, 12,  0,  0, 84],
        [77, 69, 72, 66, 58, 12, 85,  0]])
targets:
tensor([[ 1, 73, 55, 63, 58,  1, 74, 62],
        [66, 59, 74,  1, 70, 69, 70, 70],
        [61, 66, 79, 12,  0,  0, 84, 35],
        [69, 72, 66, 58, 12, 85,  0,  0]])


In [16]:
# Creating the Language Model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) #nn.Embedding is a learnable parameter
    
    def forward(self, index, targets):
        logits = self.token_embedding_table(index)
        
        return logits